In [1]:
from transformers import AutoTokenizer

In [2]:
cp = "Premier"
tok = AutoTokenizer.from_pretrained(cp)

In [6]:
inp = tok(["je voudrais avoir la liste des commandes fournisseurs est ce que ça change ?.","ceci va changer pas mal de choses"], padding=True, truncation=True, return_tensors="pt")
print(inp)

{'input_ids': tensor([[  45,   77, 5415, 4178,  318, 2485,  509,  229,  178, 1213, 1235, 2935,
         1147, 6781,  341, 2189,   32,  110],
        [1147,  354,  555, 6138, 2312, 2648,   75,  130,  331, 4703,    1,    1,
            1,    1,    1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [7]:
inp.input_ids[0]

tensor([  45,   77, 5415, 4178,  318, 2485,  509,  229,  178, 1213, 1235, 2935,
        1147, 6781,  341, 2189,   32,  110])

In [8]:
tok.decode(inp.input_ids[0])

'j ##e vou ##dr ##ais avoir la liste des commandes fournisseurs est ce que ca change ? ##.'

In [9]:
tok.decode(inp.input_ids[1])

'ce ##ci va changer pas ma ##l de ch ##oses [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
from transformers import Pipeline


class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        postprocess_kwargs = {}
        if "top_k" in kwargs:
            postprocess_kwargs["top_k"] = kwargs["top_k"]
        return preprocess_kwargs, {}, postprocess_kwargs

    def preprocess(self, inputs, maybe_arg=2):
        model_input = Tensor(inputs["input_ids"])
        print(inputs)
        print(model_input)
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        # model_inputs == {"model_input": model_input}
        print(model_inputs)
        outputs = self.model(**model_inputs)
        print(outputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        best_class = model_outputs["logits"].softmax(-1)
        print(model_outputs)
        print(best_class)
        return best_class

In [49]:
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import AutoModelForSequenceClassification

PIPELINE_REGISTRY.register_pipeline(
    "new-task",
    pipeline_class=MyPipeline
    #pt_model=AutoModelForSequenceClassification,
)


new-task is already registered. Overwriting pipeline for task new-task...


In [50]:
from transformers import pipeline
cla = pipeline("new-task", default="_forward")

KeyError: 'default'

In [26]:
from huggingface_hub import PyTorchModelHubMixin

In [38]:
model = PyTorchModelHubMixin.from_pretrained("Premier",output_attention=True, foo=False,cache_dir=os.path.dirname(notebook_path))

Loading weights from local directory


AttributeError: 'PyTorchModelHubMixin' object has no attribute 'state_dict'

In [39]:
from transformers import AutoConfig
import os

notebook_path = os.path.abspath("MPD.ipynb")

config = AutoConfig.from_pretrained('Premier', output_attention=True, foo=False,cache_dir=os.path.dirname(notebook_path))

In [40]:
config

AlbertConfig {
  "_name_or_path": "Premier",
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "finetuning_task": "mrpc",
  "format": "pt",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 32,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "output_size": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.1",
  "type_vocab_size": 2,
  "vocab_size": 50
}

In [42]:
model = PyTorchModelHubMixin.from_pretrained("Premier")

Loading weights from local directory


AttributeError: 'PyTorchModelHubMixin' object has no attribute 'state_dict'